## Задание №1

Дана функция:

$f(x)=0.5{x^2}−0.1\frac{1}{e^{-x}}+0.5 cos(2x)-2$

Необходимо выполнить ее аппроксимацию (восстановление) на интервале [-5, 5] моделью вида:

$ a(x) = w_0 + w_1*x + w_2*x^2 + w_3 * cos(2x) + w_4 * sin(2x)$

Вектор параметров 
$w = [w_0, w_1, w_2, w_3, w_4]^T$ следует искать с помощью алгоритма стохастического градиентного спуска (SGD):

$w_n = w_{n-1}−η⋅ \frac{∂L_i(w)}{∂w}$

где i - случайно выбранный образ из обучающей выборки $L_i(w)$ - квадратичная функция потерь:

$L_i(w) = (a(x_i, w) - y_i)^2 = (w^T * x_i - y_i)^2 $ 

где $ x_i = [1,x,x^2,cos(2x),sin(2x)]^T$ - вектор признаков i-го образа; 
$y_i$ - значение функции f(x) в i-й точке.

Производная этой функции может быть записана в следующем векторно-матричном виде:

$\frac{∂L_i(w)}{∂w} = 2*(w^T * x_i - y_i)*x_i^T $

Продолжите программу, в которой объявлена функция f(x) с именем func, заданы значения функции по оси абсцисс и ординат, а также начальные значения и параметры для алгоритма SGD

На каждой итерации алгоритма SGD необходимо выбирать один случайный образ k:

``k = np.random.randint(0, sz) # sz - размер выборки (массива coord_x)``

из массива coord_x и на его основе формировать вектор признаков:

$ x_k = [1,x,x^2,cos(2x),sin(2x)]^T$

И пересчитывать значение Qe - экспоненциального скользящего среднего по формуле:

$Q_e = λ * L_k(w) + (1-λ) * Q_e $

Вычисленные параметры вектора w следует сохранить в виде списка или кортежа с именем ``w``. Также нужно вычислить итоговое значение среднего эмпирического риска для обученной модели по формуле:

$$Q(a,X) = \frac{1}{n}* \sum_ {i=1}^n {L_i(w)}$$

Вычисленное значение Q(a,X) сохраните в переменной ``Q``, а последнее значение $Q_e$ - в переменной ``Qe``.

In [ ]:
import numpy as np

# исходная функция, которую нужно аппроксимировать моделью a(x)
def func(x):
    return 0.5 * x**2 - 0.1 * 1/np.exp(-x) + 0.5 * np.cos(2*x) - 2.

coord_x = np.arange(-5.0, 5.0, 0.1) # значения по оси абсцисс [-5; 5] с шагом 0.1
coord_y = func(coord_x) # значения функции по оси ординат

sz = len(coord_x)	# количество значений функций (точек)
eta = np.array([0.01, 0.001, 0.0001, 0.01, 0.01]) # шаг обучения для каждого параметра w0, w1, w2, w3, w4
w = np.array([0., 0., 0., 0., 0.]) # начальные значения параметров модели
N = 500 # число итераций алгоритма SGD
lm = 0.02 # значение параметра лямбда для вычисления скользящего экспоненциального среднего
x_i = np.column_stack((np.ones(sz), coord_x, coord_x ** 2, np.cos(2*coord_x), np.sin(2*coord_x)))                         
Qe = np.mean((x_i @ w.T - coord_y) ** 2) # начальное значение среднего эмпирического риска
np.random.seed(0) # генерация одинаковых последовательностей псевдослучайных чисел
L = np.array([])

for i in range(N):
    k = np.random.randint(0, sz-1)

    x = x_i[k]

    l = (x @ w.T - coord_y[k]) ** 2
    L = np.append(L,l)

    dl = 2 * (x @ w.T - coord_y[k]) * x.T

    w -= eta * dl

    Qe = lm * l + (1-lm) * Qe

Q = np.mean((x_i @ w.T - coord_y) ** 2) 

## Задание №2

Дана функция:

$f(x)=0.5x + 0.2x^2 - 0.05x^3 + 0.2sin(4x) - 2.5 $

Необходимо выполнить ее аппроксимацию (восстановление) на интервале [-4, 6] моделью вида:

$ a(x) = w_0 + w_1*x + w_2*x^2 + w_3 * x^3$

Вектор параметров $w = [w_0,w_1,w_2,w_3]^T$ следует искать с помощью алгоритма стохастического градиентного спуска (SGD) с квадратичной функцией потерь:

$w_n = w_{n-1}−η⋅ \frac{∂Qk(w)}{∂w}$

где k - случайно выбранный образ из обучающей выборки; Qk(w) - усеченный эмпирический риск:

$$Qk(w) = \frac{1}{K}* \sum_ {i=k}^{k+K-1} (a(x_i,w) - y_i)^2 = \frac{1}{K}* \sum_ {i=k}^{k+K-1} (w^T*x_i - y_i)^2 $$

где $x_i = [1,x,x^2,x^3]^T $ - вектор признаков i-го образа; $y_i$ - значение функции f(x) в i-й точке.

Производная усеченного показателя качества может быть записана в следующем векторно-матричном виде:

$$\frac{∂Qk(w)}{∂w} = \frac{2}{K} \sum_ {i=k}^{k+K-1} (w^T * x_i - y_i) * x_i^T $$

То есть, псевдоградиент вычисляется не по одному образу k, а по образам в диапазоне [k; k+K) (всего K образов).

Продолжите программу, в которой объявлена функция f(x) с именем func, заданы значения функции по оси абсцисс и ординат, а также начальные значения и параметры для алгоритма SGD

На каждой итерации алгоритма SGD необходимо выбирать начальное значение k командой:

``k = np.random.randint(0, sz-batch_size) # sz - размер выборки (массива coord_x)``

и из массива coord_x формировать векторы признаков:

$x_i = [1,x,x^2,x^3]^T$

для вычисления псевдоградиента Qk по образам от k до k + batch_size (команда ``range(k, k+batch_size)``).

Также на каждой итерации нужно пересчитывать значение Qe - экспоненциального скользящего среднего по формуле:

$Q_e = λ * Qk(w) + (1-λ) * Q_e $

Вычисленные параметры вектора w следует сохранить в виде списка или кортежа с именем ``w``. Также нужно вычислить итоговое значение среднего эмпирического риска для обученной модели по формуле:

$$Q(a,X) = \frac{1}{n}* \sum_ {i=1}^n {L_i(w)}$$

Вычисленное значение Q(a,X) сохраните в переменной ``Q``, а последнее значение $Q_e$ - в переменной ``Qe``.


In [ ]:
import numpy as np

# исходная функция, которую нужно аппроксимировать моделью a(x)
def func(x):
    return 0.5 * x + 0.2 * x ** 2 - 0.05 * x ** 3 + 0.2 * np.sin(4 * x) - 2.5


coord_x = np.arange(-4.0, 6.0, 0.1) # значения по оси абсцисс [-4; 6] с шагом 0.1
coord_y = func(coord_x) # значения функции по оси ординат

sz = len(coord_x)	# количество значений функций (точек)
eta = np.array([0.1, 0.01, 0.001, 0.0001]) # шаг обучения для каждого параметра w0, w1, w2, w3
w = np.array([0., 0., 0., 0.]) # начальные значения параметров модели
N = 500 # число итераций алгоритма SGD
lm = 0.02 # значение параметра лямбда для вычисления скользящего экспоненциального среднего
batch_size = 50 # размер мини-батча (величина K = 50)
x_i = np.column_stack((np.ones(sz), coord_x, coord_x ** 2, coord_x ** 3))

Qe = np.mean((x_i @ w.T - coord_y) ** 2)
np.random.seed(0) # генерация одинаковых последовательностей псевдослучайных чисел

for _ in range(N):
    k = np.random.randint(0, sz-batch_size-1)
    x_batch = x_i[k:k+batch_size]
    y_batch = coord_y[k:k+batch_size]

    y_pred = x_batch @ w.T

    error = y_pred - y_batch
    Qk = np.mean(error ** 2)  # усечённый эмпирический риск для мини-батча

    # Обновление скользящего экспоненциального среднего эмпирического риска
    Qe = lm * Qk + (1 - lm) * Qe

    er_col = error.reshape(batch_size, 1)
    # Вычисление псевдоградиента
    grad = 2 * np.mean(er_col * x_batch, axis=0)
    
    # Обновление параметров модели
    w -= eta * grad


Q = np.mean((x_i @ w.T - coord_y) ** 2)

## Задание №3

Дана функция:

$f(x) = -0.7x -0.2x^2 +0.05x^3 -0.2cos(3x) + 2$

Необходимо выполнить ее аппроксимацию (восстановление) на интервале [-4, 6] моделью вида:

$ a(x) = w_0 + w_1*x + w_2*x^2 + w_3 * x^3$

Вектор параметров $w = [w_0,w_1,w_2,w_3]^T $ следует искать с помощью алгоритма стохастического градиентного спуска (SGD) с оптимизатором импульсов Нестерова:

$v=γ⋅v+(1−γ)⋅η⋅\frac{∂Qk(w−γ⋅v)}{∂w} $

$w_n = w_{n-1} - v $

где k - случайно выбранный образ из обучающей выборки; Qk(w) - усеченный эмпирический риск:

$$Qk(w) = \frac{1}{K}* \sum_ {i=k}^{k+K-1} (a(x_i,w) - y_i)^2 = \frac{1}{K}* \sum_ {i=k}^{k+K-1} (w^T*x_i - y_i)^2 $$

где $x_i = [1,x,x^2,x^3]^T $ - вектор признаков i-го образа; $y_i$ - значение функции f(x) в i-й точке.

Производная усеченного показателя качества может быть записана в следующем векторно-матричном виде:

$$\frac{∂Qk(w−γ⋅v)}{∂w} = \frac{2}{K} \sum_ {i=k}^{k+K-1} ((w−γ⋅v)^T * x_i - y_i) * x_i^T $$

То есть, псевдоградиент вычисляется не по одному образу k, а по образам в диапазоне [k; k+K) (всего K образов).

Продолжите программу, в которой объявлена функция f(x) с именем func, заданы значения функции по оси абсцисс и ординат, а также начальные значения и параметры для алгоритма SGD с импульсами Нестерова

На каждой итерации алгоритма SGD необходимо выбирать начальное значение k командой:

``k = np.random.randint(0, sz-batch_size-1) # sz - размер выборки (массива coord_x)``

и из массива coord_x формировать векторы признаков:
$x_i = [1,x,x^2,x^3]^T$

для вычисления псевдоградиента Qk по образам от k до k + batch_size (команда ``range(k, k+batch_size)``)

Также на каждой итерации нужно пересчитывать значение Qe экспоненциального скользящего среднего по формуле:

$Q_e = λ * Qk(w) + (1-λ) * Q_e $

Вычисленные параметры вектора w следует сохранить в виде списка или кортежа с именем w. Также нужно вычислить итоговое значение среднего эмпирического риска для обученной модели по формуле:

$$Q(a,X) = \frac{1}{n}* \sum_ {i=1}^n {L_i(w)}$$

Вычисленное значение Q(a,X) сохраните в переменной ``Q``, а последнее значение Qe - в переменной ``Qe``.

In [ ]:
import numpy as np

# исходная функция, которую нужно аппроксимировать моделью a(x)
def func(x):
    return -0.7 * x - 0.2 * x ** 2 + 0.05 * x ** 3 - 0.2 * np.cos(3 * x) + 2

def a(x,w):
    return x @ w.T

coord_x = np.arange(-4.0, 6.0, 0.1) # значения по оси абсцисс [-4; 6] с шагом 0.1
coord_y = func(coord_x) # значения функции по оси ординат

sz = len(coord_x)	# количество значений функций (точек)
eta = np.array([0.1, 0.01, 0.001, 0.0001]) # шаг обучения для каждого параметра w0, w1, w2, w3
w = np.array([0., 0., 0., 0.]) # начальные значения параметров модели
N = 500 # число итераций алгоритма SGD
lm = 0.02 # значение параметра лямбда для вычисления скользящего экспоненциального среднего
batch_size = 20 # размер мини-батча (величина K = 20)
gamma = 0.8 # коэффициент гамма для вычисления импульсов Нестерова
v = np.zeros(len(w))  # начальное значение [0, 0, 0, 0]
x = np.array([(1, xx, xx ** 2, xx ** 3) for xx in coord_x])

Qe = 0
np.random.seed(0) # генерация одинаковых последовательностей псевдослучайных чисел

for _ in range(N):
    k = np.random.randint(0, sz-batch_size-1) # sz - размер выборки (массива coord_x)
    x_batch = x[k:k+batch_size]
    y_batch = coord_y[k:k+batch_size]

    Qk = np.mean([(a(x[i], w) - coord_y[i]) ** 2 for i in range(k,k+batch_size)])

    dQk = 2 * np.mean([((w - gamma * v).T @ x[i] - coord_y[i]) * x[i].T for i in range(k, k+batch_size)], axis=0)

    v = gamma * v + (1-gamma) * eta * dQk

    w -= v

    Qe = lm * Qk + (1-lm) * Qe

Q = np.mean((a(x,w) - coord_y) ** 2)